# Import required libraries
### - psycopg2 is being used to facilitate transfering the dataframe from the notebook to Redshift.
### - create_engine from the sqlalchemy library is used to form the final endpoint variable.

In [60]:
import os
import pandas as pd
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from pathlib import Path
from sqlalchemy import create_engine
import psycopg2
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

# Set Endpoint variable using Redshift cradentials.

In [61]:
# project-three-bi.clpkklnojmiu.us-east-2.redshift.amazonaws.com:5439/dev
dbname = 'dev'
host = 'project-three-bi.{endpoint}.us-east-2.redshift.amazonaws.com'
user = 'username'
password = 'password'
port = '5439'

endpoint = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"

In [62]:
my_connection = create_engine(endpoint)

# Pull in Amazon review dataset from csv file.

In [63]:
reviews_df = pd.read_csv(Path("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"))
reviews_df

id             dateAdded           dateUpdated  \
0      AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
1      AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
2      AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
3      AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
4      AVpgNzjwLJeJML43Kpxn  2015-10-30T08:59:32Z  2019-04-25T09:08:16Z   
...                     ...                   ...                   ...   
28327  AVqkIhxunnc1JgDc3kg_  2017-03-06T14:59:43Z  2019-02-23T02:49:38Z   
28328  AVqkIhxunnc1JgDc3kg_  2017-03-06T14:59:43Z  2019-02-23T02:49:38Z   
28329  AVqkIhxunnc1JgDc3kg_  2017-03-06T14:59:43Z  2019-02-23T02:49:38Z   
28330  AVqkIhxunnc1JgDc3kg_  2017-03-06T14:59:43Z  2019-02-23T02:49:38Z   
28331  AVqkIhxunnc1JgDc3kg_  2017-03-06T14:59:43Z  2019-02-23T02:49:38Z   

                                                    name  \
0      AmazonBasics AAA Performance Alkaline Batterie...   
1      AmazonBasics AAA Performance Alkaline Batterie...   
2      AmazonBasics AAA Performance Alkaline Batterie...   
3      AmazonBasics AAA Performance Alkaline Batterie...   
4      AmazonBasics AAA Performance Alkaline Batterie...   
...                                                  ...   
28327  Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...   
28328  Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...   
28329  Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...   
28330  Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...   
28331  Fire HD 8 Tablet with Alexa, 8 HD Display, 16 ...   

                       asins         brand  \
0      B00QWO9P0O,B00LH3DMUO  Amazonbasics   
1      B00QWO9P0O,B00LH3DMUO  Amazonbasics   
2      B00QWO9P0O,B00LH3DMUO  Amazonbasics   
3      B00QWO9P0O,B00LH3DMUO  Amazonbasics   
4      B00QWO9P0O,B00LH3DMUO  Amazonbasics   
...                      ...           ...   
28327             B018T075DC        Amazon   
28328             B018T075DC        Amazon   
28329             B018T075DC        Amazon   
28330             B018T075DC        Amazon   
28331             B018T075DC        Amazon   

                                              categories primaryCategories  \
0      AA,AAA,Health,Electronics,Health & Household,C...   Health & Beauty   
1      AA,AAA,Health,Electronics,Health & Household,C...   Health & Beauty   
2      AA,AAA,Health,Electronics,Health & Household,C...   Health & Beauty   
3      AA,AAA,Health,Electronics,Health & Household,C...   Health & Beauty   
4      AA,AAA,Health,Electronics,Health & Household,C...   Health & Beauty   
...                                                  ...               ...   
28327  Fire Tablets,Tablets,All Tablets,Amazon Tablet...       Electronics   
28328  Fire Tablets,Tablets,All Tablets,Amazon Tablet...       Electronics   
28329  Fire Tablets,Tablets,All Tablets,Amazon Tablet...       Electronics   
28330  Fire Tablets,Tablets,All Tablets,Amazon Tablet...       Electronics   
28331  Fire Tablets,Tablets,All Tablets,Amazon Tablet...       Electronics   

                                               imageURLs  \
0      https://images-na.ssl-images-amazon.com/images...   
1      https://images-na.ssl-images-amazon.com/images...   
2      https://images-na.ssl-images-amazon.com/images...   
3      https://images-na.ssl-images-amazon.com/images...   
4      https://images-na.ssl-images-amazon.com/images...   
...                                                  ...   
28327  https://www.upccodesearch.com/images/barcode/0...   
28328  https://www.upccodesearch.com/images/barcode/0...   
28329  https://www.upccodesearch.com/images/barcode/0...   
28330  https://www.upccodesearch.com/images/barcode/0...   
28331  https://www.upccodesearch.com/images/barcode/0...   

                                                    keys  ...  \
0      amazonbasics/hl002619,amazonbasicsaaaperforman...  ...   
1      amazonbasics/hl002619,amazonbasicsaaaperforman...  ... 

# Process review descriptions through sentiment analyzer and form new dataframe.

In [64]:
reviews = []

for ind in reviews_df.index:
    try:
        sentiment = analyzer.polarity_scores(reviews_df['reviews.text'][ind])
        category = reviews_df["primaryCategories"][ind]
        review_id = reviews_df["id"][ind]
        date = reviews_df["dateAdded"][ind][:10]
        
        reviews.append({
            "review id": review_id,
            "category": category,
            "date": date,
            "compound": sentiment["compound"],
            "positive": sentiment["pos"],
            "negative": sentiment["neg"],
            "neutral": sentiment["neu"],
        })
    
    except AttributeError:
        pass
    
review_sentiment_df = pd.DataFrame(reviews)

In [65]:
review_sentiment_df.head()

review id         category        date  compound  positive  \
0  AVpgNzjwLJeJML43Kpxn  Health & Beauty  2015-10-30   -0.6908     0.000   
1  AVpgNzjwLJeJML43Kpxn  Health & Beauty  2015-10-30    0.3612     0.172   
2  AVpgNzjwLJeJML43Kpxn  Health & Beauty  2015-10-30    0.7650     0.423   
3  AVpgNzjwLJeJML43Kpxn  Health & Beauty  2015-10-30    0.6124     0.312   
4  AVpgNzjwLJeJML43Kpxn  Health & Beauty  2015-10-30    0.6249     0.313   

   negative  neutral  
0     0.186    0.814  
1     0.000    0.828  
2     0.000    0.577  
3     0.000    0.688  
4     0.000    0.687

# Send final dataframe to Redshift database. Replace any existing data table of the same name if present.

In [66]:
review_sentiment_df.to_sql('review_sentiment_table', my_connection, index=False, if_exists='replace')